# 8th_2022

## Raw Data

In [1]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)

    Example:
    --------
    blob_url = "https://github.com/user/repo/blob/main/file.xls"
    raw_url = convert_github_url_to_raw(blob_url)
    # "https://github.com/user/repo/raw/main/file.xls"
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_8th_governor_election(file_path_or_url):
    """
    제8회 시도지사 선거 데이터를 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL

    Returns:
    --------
    dict
        각 시도별로 첫 번째 행과 '합계' 행만 포함한 데이터프레임 딕셔너리

    Example:
    --------
    # 로컬 파일
    filtered_dfs = process_8th_governor_election('시도지사선거.xlsx')

    # GitHub URL
    github_url = "https://github.com/.../blob/main/시도지사선거.xlsx"
    filtered_dfs = process_8th_governor_election(github_url)
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 먼저 header 없이 파일을 읽어서 구조 확인
    df_raw = pd.read_excel(file_path_or_url, header=None)

    # 첫 번째 행과 두 번째 행을 가져오기
    first_row = df_raw.iloc[0].fillna('_').astype(str)
    second_row = df_raw.iloc[1].fillna('_').astype(str)

    # 열 이름 생성 (첫 번째 행 + 두 번째 행)
    new_columns = []
    for i in range(len(first_row)):
        if first_row[i] == '_' and second_row[i] == '_':
            new_columns.append('_')
        elif first_row[i] == '_':
            new_columns.append(second_row[i])
        elif second_row[i] == '_':
            new_columns.append(first_row[i])
        else:
            new_columns.append(f"{first_row[i]}_{second_row[i]}")

    # 데이터프레임 재구성 (3번째 행부터 데이터로 사용)
    df = df_raw.iloc[2:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print("새로 생성된 열 이름:")
    print(new_columns[:15])  # 처음 15개만 출력
    print(f"\n총 {len(new_columns)}개의 열")

    # 첫 번째 열의 이름 가져오기
    first_column = df.columns[0]

    # 첫 번째 열의 고유값 확인
    unique_values = df[first_column].unique()
    print(f"\n첫 번째 열 '{first_column}'의 고유값들:")
    print(unique_values)
    print(f"\n총 {len(unique_values)}개의 그룹으로 분리됩니다.\n")

    # 딕셔너리로 각 그룹별 데이터프레임 저장
    grouped_dfs = {}

    # 첫 번째 열의 값별로 데이터프레임 분리
    for value in unique_values:
        # 해당 값과 일치하는 행들만 필터링
        grouped_dfs[value] = df[df[first_column] == value].copy()
        print(f"'{value}' 그룹: {len(grouped_dfs[value])}개 행")

    # 각 데이터프레임에서 첫 번째 행과 '읍면동명'이 '합계'인 행만 남기기
    filtered_dfs = {}

    for value, group_df in grouped_dfs.items():
        # 빈 데이터프레임 체크
        if len(group_df) == 0:
            print(f"'{value}' 그룹은 비어있습니다. 건너뜁니다.")
            continue

        # 첫 번째 행 가져오기
        first_row = group_df.iloc[[0]]

        # '읍면동명' 열이 '합계'인 행들 찾기
        # 첫 번째 행이 이미 '합계'일 수도 있으므로 중복 제거
        if '읍면동명' in group_df.columns:
            summary_rows = group_df[group_df['읍면동명'] == '합계']

            # 첫 번째 행과 합계 행들을 합치기 (중복 제거)
            if not summary_rows.empty:
                # 첫 번째 행이 이미 '합계'인 경우 중복 방지
                if group_df.iloc[0]['읍면동명'] == '합계':
                    filtered_df = summary_rows
                else:
                    filtered_df = pd.concat([first_row, summary_rows]).drop_duplicates()
            else:
                # '합계' 행이 없는 경우 첫 번째 행만 유지
                filtered_df = first_row
        else:
            # '읍면동명' 열이 없는 경우 첫 번째 행만 유지
            filtered_df = first_row
            print(f"경고: '{value}' 그룹에 '읍면동명' 열이 없습니다.")

        filtered_dfs[value] = filtered_df.reset_index(drop=True)
        print(f"'{value}' 그룹: 원본 {len(group_df)}개 행 → 필터링 후 {len(filtered_df)}개 행")

    print(f"\n처리 완료! 총 {len(filtered_dfs)}개의 시도별 데이터프레임이 생성되었습니다.")

    return filtered_dfs


# 사용 예시

# 방법 1: 로컬 파일 사용
# filtered_dfs = process_8th_governor_election('시도지사선거.xlsx')

# 방법 2: GitHub URL 사용
# github_url = "https://github.com/user/korean-elections/blob/main/original/Local_Elections_Governor/8th_2022/시도지사선거.xlsx"
# filtered_dfs8 = process_8th_governor_election(github_url)

# 결과 확인
# print("\n사용 가능한 시도:")
# print(list(filtered_dfs8.keys()))

# 특정 시도 데이터 확인
# print("\n서울특별시 데이터:")
# print(filtered_dfs8['서울특별시'])

In [2]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url8 = "https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/8th_2022/%EC%8B%9C%EB%8F%84%EC%A7%80%EC%82%AC%EC%84%A0%EA%B1%B0.xlsx"
dfs_8th = process_8th_governor_election(blob_url8)

새로 생성된 열 이름:
['선거구명', '구시군명', '읍면동명', '구분', '선거인수_선거인수', '투표수_선거인수', '후보자별 득표수_후보1', '후보2', '후보3', '후보4', '후보5', '후보6', '계_선거인수', '무효투표수_선거인수', '기권수_선거인수']

총 15개의 열

첫 번째 열 '선거구명'의 고유값들:
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '세종특별자치시' '경기도'
 '강원도' '충청북도' '충청남도' '전라북도' '전라남도' '경상북도' '경상남도' '제주특별자치도']

총 17개의 그룹으로 분리됩니다.

'서울특별시' 그룹: 1400개 행
'부산광역시' 그룹: 695개 행
'대구광역시' 그룹: 466개 행
'인천광역시' 그룹: 515개 행
'광주광역시' 그룹: 316개 행
'대전광역시' 그룹: 268개 행
'울산광역시' 그룹: 193개 행
'세종특별자치시' 그룹: 71개 행
'경기도' 그룹: 1890개 행
'강원도' 그룹: 654개 행
'충청북도' 그룹: 529개 행
'충청남도' 그룹: 704개 행
'전라북도' 그룹: 804개 행
'전라남도' 그룹: 1001개 행
'경상북도' 그룹: 1110개 행
'경상남도' 그룹: 1025개 행
'제주특별자치도' 그룹: 139개 행
'서울특별시' 그룹: 원본 1400개 행 → 필터링 후 26개 행
'부산광역시' 그룹: 원본 695개 행 → 필터링 후 17개 행
'대구광역시' 그룹: 원본 466개 행 → 필터링 후 9개 행
'인천광역시' 그룹: 원본 515개 행 → 필터링 후 11개 행
'광주광역시' 그룹: 원본 316개 행 → 필터링 후 6개 행
'대전광역시' 그룹: 원본 268개 행 → 필터링 후 6개 행
'울산광역시' 그룹: 원본 193개 행 → 필터링 후 6개 행
'세종특별자치시' 그룹: 원본 71개 행 → 필터링 후 2개 행
'경기도' 그룹: 원본 1890개 행 → 필터링 후 43개 행
'강원도' 그룹: 원본 654개

In [3]:
list(dfs_8th.keys())

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

## Seoul


In [4]:
seoul_8th = dfs_8th['서울특별시']
seoul_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,서울특별시,종로구,NaN,NaN,NaN,NaN,더불어민주당\n송영길,국민의힘\n오세훈,정의당\n권수정,기본소득당\n신지혜,무소속\n김광종,\n,NaN,NaN,NaN
1,서울특별시,종로구,합계,NaN,"129,816","70,657","28,327","40,145","1,151",226,167,0,"70,016",641,"59,159"
2,서울특별시,중구,합계,NaN,"112,039","60,323","23,811","34,866",696,147,125,0,"59,645",678,"51,716"
3,서울특별시,용산구,합계,NaN,"199,061","104,787","34,614","67,579","1,354",329,192,0,"104,068",719,"94,274"
4,서울특별시,성동구,합계,NaN,"251,990","139,761","51,996","84,320","1,510",350,273,0,"138,449","1,312","112,229"
5,서울특별시,광진구,합계,NaN,"305,462","157,005","62,217","90,734","1,845",500,310,0,"155,606","1,399","148,457"
6,서울특별시,동대문구,합계,NaN,"302,024","156,964","63,157","89,470","2,011",461,340,0,"155,439","1,525","145,060"
7,서울특별시,중랑구,합계,NaN,"348,762","177,701","76,660","96,447","1,870",437,361,0,"175,775","1,926","171,061"
8,서울특별시,성북구,합계,NaN,"379,123","202,235","85,825","110,962","2,698",599,413,0,"200,497","1,738","176,888"
9,서울특별시,강북구,합계,NaN,"268,130","133,390","58,962","70,747","1,515",396,305,0,"131,925","1,465","134,740"


In [5]:
seoul_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [6]:
rename_seoul = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_송영길',
    '후보2': '득표수_2_국민의힘_오세훈',
    '후보3': '득표수_3_정의당_권수정',
    '후보4': '득표수_4_기본소득당_신지혜',
    '후보5': '득표수_5_무소속_김광종',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [7]:
seoul_8th = seoul_8th.rename(columns=rename_seoul).drop(columns=['읍면동명', '구분', '후보6']).drop(index=0)
seoul_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송영길,득표수_2_국민의힘_오세훈,득표수_3_정의당_권수정,득표수_4_기본소득당_신지혜,득표수_5_무소속_김광종,득표수_계,무효투표수,기권수
1,서울특별시,종로구,"129,816","70,657","28,327","40,145","1,151",226,167,"70,016",641,"59,159"
2,서울특별시,중구,"112,039","60,323","23,811","34,866",696,147,125,"59,645",678,"51,716"
3,서울특별시,용산구,"199,061","104,787","34,614","67,579","1,354",329,192,"104,068",719,"94,274"
4,서울특별시,성동구,"251,990","139,761","51,996","84,320","1,510",350,273,"138,449","1,312","112,229"
5,서울특별시,광진구,"305,462","157,005","62,217","90,734","1,845",500,310,"155,606","1,399","148,457"
6,서울특별시,동대문구,"302,024","156,964","63,157","89,470","2,011",461,340,"155,439","1,525","145,060"
7,서울특별시,중랑구,"348,762","177,701","76,660","96,447","1,870",437,361,"175,775","1,926","171,061"
8,서울특별시,성북구,"379,123","202,235","85,825","110,962","2,698",599,413,"200,497","1,738","176,888"
9,서울특별시,강북구,"268,130","133,390","58,962","70,747","1,515",396,305,"131,925","1,465","134,740"
10,서울특별시,도봉구,"280,913","148,579","61,946","83,072","1,457",397,290,"147,162","1,417","132,334"


In [8]:
seoul_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 1 to 25
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                25 non-null     object
 1   구시군               25 non-null     object
 2   선거인수              25 non-null     object
 3   투표수               25 non-null     object
 4   득표수_1_더불어민주당_송영길  25 non-null     object
 5   득표수_2_국민의힘_오세훈    25 non-null     object
 6   득표수_3_정의당_권수정     25 non-null     object
 7   득표수_4_기본소득당_신지혜   25 non-null     object
 8   득표수_5_무소속_김광종     25 non-null     object
 9   득표수_계             25 non-null     object
 10  무효투표수             25 non-null     object
 11  기권수               25 non-null     object
dtypes: object(12)
memory usage: 2.5+ KB


In [9]:
seoul_8th = seoul_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
seoul_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송영길,득표수_2_국민의힘_오세훈,득표수_3_정의당_권수정,득표수_4_기본소득당_신지혜,득표수_5_무소속_김광종,득표수_계,무효투표수,기권수
1,서울특별시,종로구,129816,70657,28327,40145,1151,226,167,70016,641,59159
2,서울특별시,중구,112039,60323,23811,34866,696,147,125,59645,678,51716
3,서울특별시,용산구,199061,104787,34614,67579,1354,329,192,104068,719,94274
4,서울특별시,성동구,251990,139761,51996,84320,1510,350,273,138449,1312,112229
5,서울특별시,광진구,305462,157005,62217,90734,1845,500,310,155606,1399,148457
6,서울특별시,동대문구,302024,156964,63157,89470,2011,461,340,155439,1525,145060
7,서울특별시,중랑구,348762,177701,76660,96447,1870,437,361,175775,1926,171061
8,서울특별시,성북구,379123,202235,85825,110962,2698,599,413,200497,1738,176888
9,서울특별시,강북구,268130,133390,58962,70747,1515,396,305,131925,1465,134740
10,서울특별시,도봉구,280913,148579,61946,83072,1457,397,290,147162,1417,132334


In [10]:
# 수치형 열만 합계 구하기
summary_row = seoul_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '서울특별시')

# summary_row를 맨 위에 붙이기
seoul_8th_with_total = pd.concat([summary_row, seoul_8th], ignore_index=True)

In [11]:
seoul_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송영길,득표수_2_국민의힘_오세훈,득표수_3_정의당_권수정,득표수_4_기본소득당_신지혜,득표수_5_무소속_김광종,득표수_계,무효투표수,기권수
0,서울특별시,합계,8378339,4455161,1733183,2608277,53840,12619,9000,4416919,38242,3923178
1,서울특별시,종로구,129816,70657,28327,40145,1151,226,167,70016,641,59159
2,서울특별시,중구,112039,60323,23811,34866,696,147,125,59645,678,51716
3,서울특별시,용산구,199061,104787,34614,67579,1354,329,192,104068,719,94274
4,서울특별시,성동구,251990,139761,51996,84320,1510,350,273,138449,1312,112229
5,서울특별시,광진구,305462,157005,62217,90734,1845,500,310,155606,1399,148457
6,서울특별시,동대문구,302024,156964,63157,89470,2011,461,340,155439,1525,145060
7,서울특별시,중랑구,348762,177701,76660,96447,1870,437,361,175775,1926,171061
8,서울특별시,성북구,379123,202235,85825,110962,2698,599,413,200497,1738,176888
9,서울특별시,강북구,268130,133390,58962,70747,1515,396,305,131925,1465,134740


In [12]:
seoul_8th_with_total.to_csv("temp1_governor_seoul_8.csv", index=False, encoding="utf-8-sig")


## Busan


In [13]:
busan_8th = dfs_8th['부산광역시']
busan_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,부산광역시,중구,NaN,NaN,NaN,NaN,더불어민주당\n변성완,국민의힘\n박형준,정의당\n김영진,\n,\n,\n,NaN,NaN,NaN
1,부산광역시,중구,합계,NaN,"38,236","19,127","5,730","12,826",268,0,0,0,"18,824",303,"19,109"
2,부산광역시,서구,합계,NaN,"93,426","47,053","13,898","31,635",701,0,0,0,"46,234",819,"46,373"
3,부산광역시,동구,합계,NaN,"80,869","40,832","12,337","27,173",562,0,0,0,"40,072",760,"40,037"
4,부산광역시,영도구,합계,NaN,"99,395","50,231","17,359","31,130",810,0,0,0,"49,299",932,"49,164"
5,부산광역시,부산진구,합계,NaN,"313,025","149,091","48,015","97,155","2,184",0,0,0,"147,354","1,737","163,934"
6,부산광역시,동래구,합계,NaN,"234,034","116,790","36,349","77,751","1,533",0,0,0,"115,633","1,157","117,244"
7,부산광역시,남구,합계,NaN,"227,019","118,507","38,576","76,951","1,483",0,0,0,"117,010","1,497","108,512"
8,부산광역시,북구,합계,NaN,"245,787","125,984","42,927","80,065","1,598",0,0,0,"124,590","1,394","119,803"
9,부산광역시,해운대구,합계,NaN,"337,958","164,774","50,629","110,375","2,023",0,0,0,"163,027","1,747","173,184"


In [14]:
busan_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [15]:
rename_busan = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_변성완',
    '후보2': '득표수_2_국민의힘_박형준',
    '후보3': '득표수_3_정의당_김영진',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [16]:
busan_8th = busan_8th.rename(columns=rename_busan).drop(columns=['읍면동명', '구분', '후보4', '후보5', '후보6']).drop(index=0)
busan_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_변성완,득표수_2_국민의힘_박형준,득표수_3_정의당_김영진,득표수_계,무효투표수,기권수
1,부산광역시,중구,"38,236","19,127","5,730","12,826",268,"18,824",303,"19,109"
2,부산광역시,서구,"93,426","47,053","13,898","31,635",701,"46,234",819,"46,373"
3,부산광역시,동구,"80,869","40,832","12,337","27,173",562,"40,072",760,"40,037"
4,부산광역시,영도구,"99,395","50,231","17,359","31,130",810,"49,299",932,"49,164"
5,부산광역시,부산진구,"313,025","149,091","48,015","97,155","2,184","147,354","1,737","163,934"
6,부산광역시,동래구,"234,034","116,790","36,349","77,751","1,533","115,633","1,157","117,244"
7,부산광역시,남구,"227,019","118,507","38,576","76,951","1,483","117,010","1,497","108,512"
8,부산광역시,북구,"245,787","125,984","42,927","80,065","1,598","124,590","1,394","119,803"
9,부산광역시,해운대구,"337,958","164,774","50,629","110,375","2,023","163,027","1,747","173,184"
10,부산광역시,기장군,"143,871","64,268","20,721","41,534","1,082","63,337",931,"79,603"


In [17]:
busan_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 1 to 16
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                16 non-null     object
 1   구시군               16 non-null     object
 2   선거인수              16 non-null     object
 3   투표수               16 non-null     object
 4   득표수_1_더불어민주당_변성완  16 non-null     object
 5   득표수_2_국민의힘_박형준    16 non-null     object
 6   득표수_3_정의당_김영진     16 non-null     object
 7   득표수_계             16 non-null     object
 8   무효투표수             16 non-null     object
 9   기권수               16 non-null     object
dtypes: object(10)
memory usage: 1.4+ KB


In [18]:
busan_8th = busan_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
busan_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_변성완,득표수_2_국민의힘_박형준,득표수_3_정의당_김영진,득표수_계,무효투표수,기권수
1,부산광역시,중구,38236,19127,5730,12826,268,18824,303,19109
2,부산광역시,서구,93426,47053,13898,31635,701,46234,819,46373
3,부산광역시,동구,80869,40832,12337,27173,562,40072,760,40037
4,부산광역시,영도구,99395,50231,17359,31130,810,49299,932,49164
5,부산광역시,부산진구,313025,149091,48015,97155,2184,147354,1737,163934
6,부산광역시,동래구,234034,116790,36349,77751,1533,115633,1157,117244
7,부산광역시,남구,227019,118507,38576,76951,1483,117010,1497,108512
8,부산광역시,북구,245787,125984,42927,80065,1598,124590,1394,119803
9,부산광역시,해운대구,337958,164774,50629,110375,2023,163027,1747,173184
10,부산광역시,기장군,143871,64268,20721,41534,1082,63337,931,79603


In [19]:
# 수치형 열만 합계 구하기
summary_row = busan_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '부산광역시')

# summary_row를 맨 위에 붙이기
busan_8th_with_total = pd.concat([summary_row, busan_8th], ignore_index=True)

In [20]:
busan_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_변성완,득표수_2_국민의힘_박형준,득표수_3_정의당_김영진,득표수_계,무효투표수,기권수
0,부산광역시,합계,2916832,1432194,455901,938601,19733,1414235,17959,1484638
1,부산광역시,중구,38236,19127,5730,12826,268,18824,303,19109
2,부산광역시,서구,93426,47053,13898,31635,701,46234,819,46373
3,부산광역시,동구,80869,40832,12337,27173,562,40072,760,40037
4,부산광역시,영도구,99395,50231,17359,31130,810,49299,932,49164
5,부산광역시,부산진구,313025,149091,48015,97155,2184,147354,1737,163934
6,부산광역시,동래구,234034,116790,36349,77751,1533,115633,1157,117244
7,부산광역시,남구,227019,118507,38576,76951,1483,117010,1497,108512
8,부산광역시,북구,245787,125984,42927,80065,1598,124590,1394,119803
9,부산광역시,해운대구,337958,164774,50629,110375,2023,163027,1747,173184


In [21]:
busan_8th_with_total.to_csv("temp1_governor_busan_8.csv", index=False, encoding="utf-8-sig")


## Daegu


In [22]:
daegu_8th = dfs_8th['대구광역시']
daegu_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,대구광역시,중구,NaN,NaN,NaN,NaN,더불어민주당\n서재헌,국민의힘\n홍준표,정의당\n한민정,기본소득당\n신원호,\n,\n,NaN,NaN,NaN
1,대구광역시,중구,합계,NaN,"68,825","29,536","5,610","22,491",746,268,0,0,"29,115",421,"39,289"
2,대구광역시,동구,합계,NaN,"297,073","131,007","24,745","100,305","2,851","1,048",0,0,"128,949","2,058","166,066"
3,대구광역시,서구,합계,NaN,"148,238","66,442","8,993","54,059","1,484",623,0,0,"65,159","1,283","81,796"
4,대구광역시,남구,합계,NaN,"129,053","54,686","8,593","43,386","1,252",554,0,0,"53,785",901,"74,367"
5,대구광역시,북구,합계,NaN,"373,423","160,433","28,966","123,952","3,864","1,356",0,0,"158,138","2,295","212,990"
6,대구광역시,수성구,합계,NaN,"349,048","157,513","29,601","120,189","3,896","1,305",0,0,"154,991","2,522","191,535"
7,대구광역시,달서구,합계,NaN,"464,339","192,191","34,133","149,512","4,674","1,531",0,0,"189,850","2,341","272,148"
8,대구광역시,달성군,합계,NaN,"214,580","91,333","15,788","71,265","2,137",857,0,0,"90,047","1,286","123,247"


In [23]:
daegu_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [24]:
rename_daegu = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_서재헌',
    '후보2': '득표수_2_국민의힘_홍준표',
    '후보3': '득표수_3_정의당_한민정',
    '후보4': '득표수_4_기본소득당_신원호',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [25]:
daegu_8th = daegu_8th.rename(columns=rename_daegu).drop(columns=['읍면동명', '구분', '후보5', '후보6']).drop(index=0)
daegu_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_서재헌,득표수_2_국민의힘_홍준표,득표수_3_정의당_한민정,득표수_4_기본소득당_신원호,득표수_계,무효투표수,기권수
1,대구광역시,중구,"68,825","29,536","5,610","22,491",746,268,"29,115",421,"39,289"
2,대구광역시,동구,"297,073","131,007","24,745","100,305","2,851","1,048","128,949","2,058","166,066"
3,대구광역시,서구,"148,238","66,442","8,993","54,059","1,484",623,"65,159","1,283","81,796"
4,대구광역시,남구,"129,053","54,686","8,593","43,386","1,252",554,"53,785",901,"74,367"
5,대구광역시,북구,"373,423","160,433","28,966","123,952","3,864","1,356","158,138","2,295","212,990"
6,대구광역시,수성구,"349,048","157,513","29,601","120,189","3,896","1,305","154,991","2,522","191,535"
7,대구광역시,달서구,"464,339","192,191","34,133","149,512","4,674","1,531","189,850","2,341","272,148"
8,대구광역시,달성군,"214,580","91,333","15,788","71,265","2,137",857,"90,047","1,286","123,247"


In [26]:
daegu_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 1 to 8
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                8 non-null      object
 1   구시군               8 non-null      object
 2   선거인수              8 non-null      object
 3   투표수               8 non-null      object
 4   득표수_1_더불어민주당_서재헌  8 non-null      object
 5   득표수_2_국민의힘_홍준표    8 non-null      object
 6   득표수_3_정의당_한민정     8 non-null      object
 7   득표수_4_기본소득당_신원호   8 non-null      object
 8   득표수_계             8 non-null      object
 9   무효투표수             8 non-null      object
 10  기권수               8 non-null      object
dtypes: object(11)
memory usage: 836.0+ bytes


In [27]:
daegu_8th = daegu_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
daegu_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_서재헌,득표수_2_국민의힘_홍준표,득표수_3_정의당_한민정,득표수_4_기본소득당_신원호,득표수_계,무효투표수,기권수
1,대구광역시,중구,68825,29536,5610,22491,746,268,29115,421,39289
2,대구광역시,동구,297073,131007,24745,100305,2851,1048,128949,2058,166066
3,대구광역시,서구,148238,66442,8993,54059,1484,623,65159,1283,81796
4,대구광역시,남구,129053,54686,8593,43386,1252,554,53785,901,74367
5,대구광역시,북구,373423,160433,28966,123952,3864,1356,158138,2295,212990
6,대구광역시,수성구,349048,157513,29601,120189,3896,1305,154991,2522,191535
7,대구광역시,달서구,464339,192191,34133,149512,4674,1531,189850,2341,272148
8,대구광역시,달성군,214580,91333,15788,71265,2137,857,90047,1286,123247


In [28]:
# 수치형 열만 합계 구하기
summary_row = daegu_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대구광역시')

# summary_row를 맨 위에 붙이기
daegu_8th_with_total = pd.concat([summary_row, daegu_8th], ignore_index=True)

In [29]:
daegu_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_서재헌,득표수_2_국민의힘_홍준표,득표수_3_정의당_한민정,득표수_4_기본소득당_신원호,득표수_계,무효투표수,기권수
0,대구광역시,합계,2044579,883141,156429,685159,20904,7542,870034,13107,1161438
1,대구광역시,중구,68825,29536,5610,22491,746,268,29115,421,39289
2,대구광역시,동구,297073,131007,24745,100305,2851,1048,128949,2058,166066
3,대구광역시,서구,148238,66442,8993,54059,1484,623,65159,1283,81796
4,대구광역시,남구,129053,54686,8593,43386,1252,554,53785,901,74367
5,대구광역시,북구,373423,160433,28966,123952,3864,1356,158138,2295,212990
6,대구광역시,수성구,349048,157513,29601,120189,3896,1305,154991,2522,191535
7,대구광역시,달서구,464339,192191,34133,149512,4674,1531,189850,2341,272148
8,대구광역시,달성군,214580,91333,15788,71265,2137,857,90047,1286,123247


In [30]:
daegu_8th_with_total.to_csv("temp1_governor_daegu_8.csv", index=False, encoding="utf-8-sig")


## Incheon


In [31]:
incheon_8th = dfs_8th['인천광역시']
incheon_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,인천광역시,중구,NaN,NaN,NaN,NaN,더불어민주당\n박남춘,국민의힘\n유정복,정의당\n이정미,기본소득당\n김한별,\n,\n,NaN,NaN,NaN
1,인천광역시,중구,합계,NaN,"124,974","57,362","24,425","30,190","1,616",294,0,0,"56,525",837,"67,612"
2,인천광역시,동구,합계,NaN,"52,728","28,942","11,592","15,595","1,011",207,0,0,"28,405",537,"23,786"
3,인천광역시,미추홀구,합계,NaN,"358,612","160,328","66,673","86,217","4,487",890,0,0,"158,267","2,061","198,284"
4,인천광역시,연수구,합계,NaN,"317,883","164,269","63,175","90,120","9,133",514,0,0,"162,942","1,327","153,614"
5,인천광역시,남동구,합계,NaN,"441,226","212,064","96,632","106,526","5,647",873,0,0,"209,678","2,386","229,162"
6,인천광역시,부평구,합계,NaN,"426,463","202,623","94,474","99,177","5,740",953,0,0,"200,344","2,279","223,840"
7,인천광역시,계양구,합계,NaN,"258,156","144,721","72,090","66,117","4,158",733,0,0,"143,098","1,623","113,435"
8,인천광역시,서구,합계,NaN,"472,254","218,379","101,408","107,661","5,832","1,001",0,0,"215,902","2,477","253,875"
9,인천광역시,강화군,합계,NaN,"63,147","39,088","11,660","24,697","1,002",444,0,0,"37,803","1,285","24,059"


In [32]:
incheon_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [33]:
rename_incheon = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_박남춘',
    '후보2': '득표수_2_국민의힘_유정복',
    '후보3': '득표수_3_정의당_이정미',
    '후보4': '득표수_4_기본소득당_김한별',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [34]:
incheon_8th = incheon_8th.rename(columns=rename_incheon).drop(columns=['읍면동명', '구분', '후보5', '후보6']).drop(index=0)
incheon_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_국민의힘_유정복,득표수_3_정의당_이정미,득표수_4_기본소득당_김한별,득표수_계,무효투표수,기권수
1,인천광역시,중구,"124,974","57,362","24,425","30,190","1,616",294,"56,525",837,"67,612"
2,인천광역시,동구,"52,728","28,942","11,592","15,595","1,011",207,"28,405",537,"23,786"
3,인천광역시,미추홀구,"358,612","160,328","66,673","86,217","4,487",890,"158,267","2,061","198,284"
4,인천광역시,연수구,"317,883","164,269","63,175","90,120","9,133",514,"162,942","1,327","153,614"
5,인천광역시,남동구,"441,226","212,064","96,632","106,526","5,647",873,"209,678","2,386","229,162"
6,인천광역시,부평구,"426,463","202,623","94,474","99,177","5,740",953,"200,344","2,279","223,840"
7,인천광역시,계양구,"258,156","144,721","72,090","66,117","4,158",733,"143,098","1,623","113,435"
8,인천광역시,서구,"472,254","218,379","101,408","107,661","5,832","1,001","215,902","2,477","253,875"
9,인천광역시,강화군,"63,147","39,088","11,660","24,697","1,002",444,"37,803","1,285","24,059"
10,인천광역시,옹진군,"18,895","12,693","3,756","7,950",295,170,"12,171",522,"6,202"


In [35]:
incheon_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                10 non-null     object
 1   구시군               10 non-null     object
 2   선거인수              10 non-null     object
 3   투표수               10 non-null     object
 4   득표수_1_더불어민주당_박남춘  10 non-null     object
 5   득표수_2_국민의힘_유정복    10 non-null     object
 6   득표수_3_정의당_이정미     10 non-null     object
 7   득표수_4_기본소득당_김한별   10 non-null     object
 8   득표수_계             10 non-null     object
 9   무효투표수             10 non-null     object
 10  기권수               10 non-null     object
dtypes: object(11)
memory usage: 1012.0+ bytes


In [36]:
incheon_8th = incheon_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
incheon_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_국민의힘_유정복,득표수_3_정의당_이정미,득표수_4_기본소득당_김한별,득표수_계,무효투표수,기권수
1,인천광역시,중구,124974,57362,24425,30190,1616,294,56525,837,67612
2,인천광역시,동구,52728,28942,11592,15595,1011,207,28405,537,23786
3,인천광역시,미추홀구,358612,160328,66673,86217,4487,890,158267,2061,198284
4,인천광역시,연수구,317883,164269,63175,90120,9133,514,162942,1327,153614
5,인천광역시,남동구,441226,212064,96632,106526,5647,873,209678,2386,229162
6,인천광역시,부평구,426463,202623,94474,99177,5740,953,200344,2279,223840
7,인천광역시,계양구,258156,144721,72090,66117,4158,733,143098,1623,113435
8,인천광역시,서구,472254,218379,101408,107661,5832,1001,215902,2477,253875
9,인천광역시,강화군,63147,39088,11660,24697,1002,444,37803,1285,24059
10,인천광역시,옹진군,18895,12693,3756,7950,295,170,12171,522,6202


In [37]:
# 수치형 열만 합계 구하기
summary_row = incheon_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '인천광역시')

# summary_row를 맨 위에 붙이기
incheon_8th_with_total = pd.concat([summary_row, incheon_8th], ignore_index=True)

In [38]:
incheon_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_국민의힘_유정복,득표수_3_정의당_이정미,득표수_4_기본소득당_김한별,득표수_계,무효투표수,기권수
0,인천광역시,합계,2534338,1240469,545885,634250,38921,6079,1225135,15334,1293869
1,인천광역시,중구,124974,57362,24425,30190,1616,294,56525,837,67612
2,인천광역시,동구,52728,28942,11592,15595,1011,207,28405,537,23786
3,인천광역시,미추홀구,358612,160328,66673,86217,4487,890,158267,2061,198284
4,인천광역시,연수구,317883,164269,63175,90120,9133,514,162942,1327,153614
5,인천광역시,남동구,441226,212064,96632,106526,5647,873,209678,2386,229162
6,인천광역시,부평구,426463,202623,94474,99177,5740,953,200344,2279,223840
7,인천광역시,계양구,258156,144721,72090,66117,4158,733,143098,1623,113435
8,인천광역시,서구,472254,218379,101408,107661,5832,1001,215902,2477,253875
9,인천광역시,강화군,63147,39088,11660,24697,1002,444,37803,1285,24059


In [39]:
incheon_8th_with_total.to_csv("temp1_governor_incheon_8.csv", index=False, encoding="utf-8-sig")


## Gwangju


In [40]:
gwangju_8th = dfs_8th['광주광역시']
gwangju_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,광주광역시,동구,NaN,NaN,NaN,NaN,더불어민주당\n강기정,국민의힘\n주기환,정의당\n장연주,기본소득당\n문현철,진보당\n김주업,\n,NaN,NaN,NaN
1,광주광역시,동구,합계,NaN,"90,255","38,373","27,775","6,994","1,468",299,"1,197",0,"37,733",640,"51,882"
2,광주광역시,서구,합계,NaN,"246,841","97,941","71,873","15,552","4,728",645,"3,542",0,"96,340","1,601","148,900"
3,광주광역시,남구,합계,NaN,"180,158","71,496","53,512","11,406","2,808",459,"2,138",0,"70,323","1,173","108,662"
4,광주광역시,북구,합계,NaN,"363,222","137,865","102,468","21,054","6,213","1,037","4,831",0,"135,603","2,262","225,357"
5,광주광역시,광산구,합계,NaN,"326,410","108,841","79,071","16,056","5,853",904,"4,887",0,"106,771","2,070","217,569"


In [41]:
gwangju_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [42]:
rename_gwangju = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_강기정',
    '후보2': '득표수_2_국민의힘_주기환',
    '후보3': '득표수_3_정의당_장연주',
    '후보4': '득표수_4_기본소득당_문현철',
    '후보5': '득표수_5_진보당_김주업',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [43]:
gwangju_8th = gwangju_8th.rename(columns=rename_gwangju).drop(columns=['읍면동명', '구분', '후보6']).drop(index=0)
gwangju_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_강기정,득표수_2_국민의힘_주기환,득표수_3_정의당_장연주,득표수_4_기본소득당_문현철,득표수_5_진보당_김주업,득표수_계,무효투표수,기권수
1,광주광역시,동구,"90,255","38,373","27,775","6,994","1,468",299,"1,197","37,733",640,"51,882"
2,광주광역시,서구,"246,841","97,941","71,873","15,552","4,728",645,"3,542","96,340","1,601","148,900"
3,광주광역시,남구,"180,158","71,496","53,512","11,406","2,808",459,"2,138","70,323","1,173","108,662"
4,광주광역시,북구,"363,222","137,865","102,468","21,054","6,213","1,037","4,831","135,603","2,262","225,357"
5,광주광역시,광산구,"326,410","108,841","79,071","16,056","5,853",904,"4,887","106,771","2,070","217,569"


In [44]:
gwangju_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 1 to 5
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                5 non-null      object
 1   구시군               5 non-null      object
 2   선거인수              5 non-null      object
 3   투표수               5 non-null      object
 4   득표수_1_더불어민주당_강기정  5 non-null      object
 5   득표수_2_국민의힘_주기환    5 non-null      object
 6   득표수_3_정의당_장연주     5 non-null      object
 7   득표수_4_기본소득당_문현철   5 non-null      object
 8   득표수_5_진보당_김주업     5 non-null      object
 9   득표수_계             5 non-null      object
 10  무효투표수             5 non-null      object
 11  기권수               5 non-null      object
dtypes: object(12)
memory usage: 612.0+ bytes


In [45]:
gwangju_8th = gwangju_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gwangju_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_강기정,득표수_2_국민의힘_주기환,득표수_3_정의당_장연주,득표수_4_기본소득당_문현철,득표수_5_진보당_김주업,득표수_계,무효투표수,기권수
1,광주광역시,동구,90255,38373,27775,6994,1468,299,1197,37733,640,51882
2,광주광역시,서구,246841,97941,71873,15552,4728,645,3542,96340,1601,148900
3,광주광역시,남구,180158,71496,53512,11406,2808,459,2138,70323,1173,108662
4,광주광역시,북구,363222,137865,102468,21054,6213,1037,4831,135603,2262,225357
5,광주광역시,광산구,326410,108841,79071,16056,5853,904,4887,106771,2070,217569


In [46]:
# 수치형 열만 합계 구하기
summary_row = gwangju_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '광주광역시')

# summary_row를 맨 위에 붙이기
gwangju_8th_with_total = pd.concat([summary_row, gwangju_8th], ignore_index=True)

In [47]:
gwangju_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_강기정,득표수_2_국민의힘_주기환,득표수_3_정의당_장연주,득표수_4_기본소득당_문현철,득표수_5_진보당_김주업,득표수_계,무효투표수,기권수
0,광주광역시,합계,1206886,454516,334699,71062,21070,3344,16595,446770,7746,752370
1,광주광역시,동구,90255,38373,27775,6994,1468,299,1197,37733,640,51882
2,광주광역시,서구,246841,97941,71873,15552,4728,645,3542,96340,1601,148900
3,광주광역시,남구,180158,71496,53512,11406,2808,459,2138,70323,1173,108662
4,광주광역시,북구,363222,137865,102468,21054,6213,1037,4831,135603,2262,225357
5,광주광역시,광산구,326410,108841,79071,16056,5853,904,4887,106771,2070,217569


In [48]:
gwangju_8th_with_total.to_csv("temp1_governor_gwangju_8.csv", index=False, encoding="utf-8-sig")


## Daejeon


In [49]:
daejeon_8th = dfs_8th['대전광역시']
daejeon_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,대전광역시,동구,NaN,NaN,NaN,NaN,더불어민주당\n허태정,국민의힘\n이장우,\n,\n,\n,\n,NaN,NaN,NaN
1,대전광역시,동구,합계,NaN,"191,882","94,041","42,619","50,120",0,0,0,0,"92,739","1,302","97,841"
2,대전광역시,중구,합계,NaN,"199,894","100,343","45,434","53,666",0,0,0,0,"99,100","1,243","99,551"
3,대전광역시,서구,합계,NaN,"399,035","192,546","93,222","97,149",0,0,0,0,"190,371","2,175","206,489"
4,대전광역시,유성구,합계,NaN,"289,980","149,576","78,142","70,109",0,0,0,0,"148,251","1,325","140,404"
5,대전광역시,대덕구,합계,NaN,"152,766","76,133","36,138","38,991",0,0,0,0,"75,129","1,004","76,633"


In [50]:
daejeon_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [51]:
rename_daejeon = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_허태정',
    '후보2': '득표수_2_국민의힘_이장우',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [52]:
daejeon_8th = daejeon_8th.rename(columns=rename_daejeon).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
daejeon_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_국민의힘_이장우,득표수_계,무효투표수,기권수
1,대전광역시,동구,"191,882","94,041","42,619","50,120","92,739","1,302","97,841"
2,대전광역시,중구,"199,894","100,343","45,434","53,666","99,100","1,243","99,551"
3,대전광역시,서구,"399,035","192,546","93,222","97,149","190,371","2,175","206,489"
4,대전광역시,유성구,"289,980","149,576","78,142","70,109","148,251","1,325","140,404"
5,대전광역시,대덕구,"152,766","76,133","36,138","38,991","75,129","1,004","76,633"


In [53]:
daejeon_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 1 to 5
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                5 non-null      object
 1   구시군               5 non-null      object
 2   선거인수              5 non-null      object
 3   투표수               5 non-null      object
 4   득표수_1_더불어민주당_허태정  5 non-null      object
 5   득표수_2_국민의힘_이장우    5 non-null      object
 6   득표수_계             5 non-null      object
 7   무효투표수             5 non-null      object
 8   기권수               5 non-null      object
dtypes: object(9)
memory usage: 492.0+ bytes


In [54]:
daejeon_8th = daejeon_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
daejeon_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_국민의힘_이장우,득표수_계,무효투표수,기권수
1,대전광역시,동구,191882,94041,42619,50120,92739,1302,97841
2,대전광역시,중구,199894,100343,45434,53666,99100,1243,99551
3,대전광역시,서구,399035,192546,93222,97149,190371,2175,206489
4,대전광역시,유성구,289980,149576,78142,70109,148251,1325,140404
5,대전광역시,대덕구,152766,76133,36138,38991,75129,1004,76633


In [55]:
# 수치형 열만 합계 구하기
summary_row = daejeon_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대전광역시')

# summary_row를 맨 위에 붙이기
daejeon_8th_with_total = pd.concat([summary_row, daejeon_8th], ignore_index=True)

In [56]:
daejeon_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_국민의힘_이장우,득표수_계,무효투표수,기권수
0,대전광역시,합계,1233557,612639,295555,310035,605590,7049,620918
1,대전광역시,동구,191882,94041,42619,50120,92739,1302,97841
2,대전광역시,중구,199894,100343,45434,53666,99100,1243,99551
3,대전광역시,서구,399035,192546,93222,97149,190371,2175,206489
4,대전광역시,유성구,289980,149576,78142,70109,148251,1325,140404
5,대전광역시,대덕구,152766,76133,36138,38991,75129,1004,76633


In [57]:
daejeon_8th_with_total.to_csv("temp1_governor_daejeon_8.csv", index=False, encoding="utf-8-sig")


## Ulsan


In [58]:
ulsan_8th = dfs_8th['울산광역시']
ulsan_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,울산광역시,중구,NaN,NaN,NaN,NaN,더불어민주당\n송철호,국민의힘\n김두겸,\n,\n,\n,\n,NaN,NaN,NaN
1,울산광역시,중구,합계,NaN,"181,055","98,391","36,596","60,730",0,0,0,0,"97,326","1,065","82,664"
2,울산광역시,남구,합계,NaN,"267,930","134,765","48,425","84,987",0,0,0,0,"133,412","1,353","133,165"
3,울산광역시,동구,합계,NaN,"127,369","70,833","32,032","37,751",0,0,0,0,"69,783","1,050","56,536"
4,울산광역시,북구,합계,NaN,"175,784","88,502","41,248","46,234",0,0,0,0,"87,482","1,020","87,282"
5,울산광역시,울주군,합계,NaN,"189,051","99,375","37,129","60,861",0,0,0,0,"97,990","1,385","89,676"


In [59]:
ulsan_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [60]:
rename_ulsan = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_송철호',
    '후보2': '득표수_2_국민의힘_김두겸',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [61]:
ulsan_8th = ulsan_8th.rename(columns=rename_ulsan).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
ulsan_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_국민의힘_김두겸,득표수_계,무효투표수,기권수
1,울산광역시,중구,"181,055","98,391","36,596","60,730","97,326","1,065","82,664"
2,울산광역시,남구,"267,930","134,765","48,425","84,987","133,412","1,353","133,165"
3,울산광역시,동구,"127,369","70,833","32,032","37,751","69,783","1,050","56,536"
4,울산광역시,북구,"175,784","88,502","41,248","46,234","87,482","1,020","87,282"
5,울산광역시,울주군,"189,051","99,375","37,129","60,861","97,990","1,385","89,676"


In [62]:
ulsan_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 1 to 5
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                5 non-null      object
 1   구시군               5 non-null      object
 2   선거인수              5 non-null      object
 3   투표수               5 non-null      object
 4   득표수_1_더불어민주당_송철호  5 non-null      object
 5   득표수_2_국민의힘_김두겸    5 non-null      object
 6   득표수_계             5 non-null      object
 7   무효투표수             5 non-null      object
 8   기권수               5 non-null      object
dtypes: object(9)
memory usage: 492.0+ bytes


In [63]:
ulsan_8th = ulsan_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
ulsan_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_국민의힘_김두겸,득표수_계,무효투표수,기권수
1,울산광역시,중구,181055,98391,36596,60730,97326,1065,82664
2,울산광역시,남구,267930,134765,48425,84987,133412,1353,133165
3,울산광역시,동구,127369,70833,32032,37751,69783,1050,56536
4,울산광역시,북구,175784,88502,41248,46234,87482,1020,87282
5,울산광역시,울주군,189051,99375,37129,60861,97990,1385,89676


In [64]:
# 수치형 열만 합계 구하기
summary_row = ulsan_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '울산광역시')

# summary_row를 맨 위에 붙이기
ulsan_8th_with_total = pd.concat([summary_row, ulsan_8th], ignore_index=True)

In [65]:
ulsan_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_국민의힘_김두겸,득표수_계,무효투표수,기권수
0,울산광역시,합계,941189,491866,195430,290563,485993,5873,449323
1,울산광역시,중구,181055,98391,36596,60730,97326,1065,82664
2,울산광역시,남구,267930,134765,48425,84987,133412,1353,133165
3,울산광역시,동구,127369,70833,32032,37751,69783,1050,56536
4,울산광역시,북구,175784,88502,41248,46234,87482,1020,87282
5,울산광역시,울주군,189051,99375,37129,60861,97990,1385,89676


In [66]:
ulsan_8th_with_total.to_csv("temp1_governor_ulsan_8.csv", index=False, encoding="utf-8-sig")


## Sejong


In [67]:
sejong_8th = dfs_8th['세종특별자치시']
sejong_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,세종특별자치시,세종특별자치시,NaN,NaN,NaN,NaN,더불어민주당\n이춘희,국민의힘\n최민호,\n,\n,\n,\n,NaN,NaN,NaN
1,세종특별자치시,세종특별자치시,합계,NaN,"292,259","149,751","69,995","78,415",0,0,0,0,"148,410","1,341","142,508"


In [68]:
sejong_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [69]:
rename_sejong = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_이춘희',
    '후보2': '득표수_2_국민의힘_최민호',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [70]:
sejong_8th = sejong_8th.rename(columns=rename_sejong).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
sejong_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_국민의힘_최민호,득표수_계,무효투표수,기권수
1,세종특별자치시,세종특별자치시,"292,259","149,751","69,995","78,415","148,410","1,341","142,508"


In [71]:
sejong_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 1 to 1
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                1 non-null      object
 1   구시군               1 non-null      object
 2   선거인수              1 non-null      object
 3   투표수               1 non-null      object
 4   득표수_1_더불어민주당_이춘희  1 non-null      object
 5   득표수_2_국민의힘_최민호    1 non-null      object
 6   득표수_계             1 non-null      object
 7   무효투표수             1 non-null      object
 8   기권수               1 non-null      object
dtypes: object(9)
memory usage: 204.0+ bytes


In [72]:
sejong_8th = sejong_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
sejong_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_국민의힘_최민호,득표수_계,무효투표수,기권수
1,세종특별자치시,세종특별자치시,292259,149751,69995,78415,148410,1341,142508


In [73]:
sejong_8th_with_total = sejong_8th.assign(구시군='합계')

In [74]:
sejong_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_국민의힘_최민호,득표수_계,무효투표수,기권수
1,세종특별자치시,합계,292259,149751,69995,78415,148410,1341,142508


In [75]:
sejong_8th_with_total.to_csv("temp1_governor_sejong_8.csv", index=False, encoding="utf-8-sig")


## Gyeonggi


In [76]:
gyeonggi_8th = dfs_8th['경기도']
gyeonggi_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,경기도,수원시장안구,NaN,NaN,NaN,NaN,더불어민주당\n김동연,국민의힘\n김은혜,정의당\n황순식,기본소득당\n서태성,진보당\n송영주,무소속\n강용석,NaN,NaN,NaN
1,경기도,수원시장안구,합계,NaN,"199,204","106,092","53,333","49,817",627,153,259,869,"105,058","1,034","93,112"
2,경기도,수원시권선구,합계,NaN,"352,407","173,856","88,562","80,304","1,090",312,406,"1,580","172,254","1,602","178,551"
3,경기도,수원시팔달구,합계,NaN,"164,086","79,283","37,937","38,931",488,169,176,724,"78,425",858,"84,803"
4,경기도,수원시영통구,합계,NaN,"296,856","160,556","81,109","75,184","1,142",212,223,"1,647","159,517","1,039","136,300"
5,경기도,성남시수정구,합계,NaN,"208,907","106,922","53,684","49,669",630,250,493,964,"105,690","1,232","101,985"
6,경기도,성남시중원구,합계,NaN,"185,468","94,477","49,268","41,860",623,212,576,820,"93,359","1,118","90,991"
7,경기도,성남시분당구,합계,NaN,"404,133","250,683","104,254","140,622","1,266",197,265,"2,677","249,281","1,402","153,450"
8,경기도,의정부시,합계,NaN,"400,177","187,916","92,465","89,731","1,241",259,584,"1,742","186,022","1,894","212,261"
9,경기도,안양시만안구,합계,NaN,"208,936","111,050","54,630","53,271",712,157,199,904,"109,873","1,177","97,886"


In [77]:
gyeonggi_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [78]:
rename_gyeonggi = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_김동연',
    '후보2': '득표수_2_국민의힘_김은혜',
    '후보3': '득표수_3_정의당_황순식',
    '후보4': '득표수_4_기본소득당_서태성',
    '후보5': '득표수_5_진보당_송영주',
    '후보6': '득표수_6_무소속_강용석',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [79]:
gyeonggi_8th = gyeonggi_8th.rename(columns=rename_gyeonggi).drop(columns=['읍면동명', '구분']).drop(index=0)
gyeonggi_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김동연,득표수_2_국민의힘_김은혜,득표수_3_정의당_황순식,득표수_4_기본소득당_서태성,득표수_5_진보당_송영주,득표수_6_무소속_강용석,득표수_계,무효투표수,기권수
1,경기도,수원시장안구,"199,204","106,092","53,333","49,817",627,153,259,869,"105,058","1,034","93,112"
2,경기도,수원시권선구,"352,407","173,856","88,562","80,304","1,090",312,406,"1,580","172,254","1,602","178,551"
3,경기도,수원시팔달구,"164,086","79,283","37,937","38,931",488,169,176,724,"78,425",858,"84,803"
4,경기도,수원시영통구,"296,856","160,556","81,109","75,184","1,142",212,223,"1,647","159,517","1,039","136,300"
5,경기도,성남시수정구,"208,907","106,922","53,684","49,669",630,250,493,964,"105,690","1,232","101,985"
6,경기도,성남시중원구,"185,468","94,477","49,268","41,860",623,212,576,820,"93,359","1,118","90,991"
7,경기도,성남시분당구,"404,133","250,683","104,254","140,622","1,266",197,265,"2,677","249,281","1,402","153,450"
8,경기도,의정부시,"400,177","187,916","92,465","89,731","1,241",259,584,"1,742","186,022","1,894","212,261"
9,경기도,안양시만안구,"208,936","111,050","54,630","53,271",712,157,199,904,"109,873","1,177","97,886"
10,경기도,안양시동안구,"265,101","155,858","77,230","74,669","1,001",163,186,"1,432","154,681","1,177","109,243"


In [80]:
gyeonggi_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 1 to 42
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                42 non-null     object
 1   구시군               42 non-null     object
 2   선거인수              42 non-null     object
 3   투표수               42 non-null     object
 4   득표수_1_더불어민주당_김동연  42 non-null     object
 5   득표수_2_국민의힘_김은혜    42 non-null     object
 6   득표수_3_정의당_황순식     42 non-null     object
 7   득표수_4_기본소득당_서태성   42 non-null     object
 8   득표수_5_진보당_송영주     42 non-null     object
 9   득표수_6_무소속_강용석     42 non-null     object
 10  득표수_계             42 non-null     object
 11  무효투표수             42 non-null     object
 12  기권수               42 non-null     object
dtypes: object(13)
memory usage: 4.4+ KB


In [81]:
gyeonggi_8th = gyeonggi_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gyeonggi_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김동연,득표수_2_국민의힘_김은혜,득표수_3_정의당_황순식,득표수_4_기본소득당_서태성,득표수_5_진보당_송영주,득표수_6_무소속_강용석,득표수_계,무효투표수,기권수
1,경기도,수원시장안구,199204,106092,53333,49817,627,153,259,869,105058,1034,93112
2,경기도,수원시권선구,352407,173856,88562,80304,1090,312,406,1580,172254,1602,178551
3,경기도,수원시팔달구,164086,79283,37937,38931,488,169,176,724,78425,858,84803
4,경기도,수원시영통구,296856,160556,81109,75184,1142,212,223,1647,159517,1039,136300
5,경기도,성남시수정구,208907,106922,53684,49669,630,250,493,964,105690,1232,101985
6,경기도,성남시중원구,185468,94477,49268,41860,623,212,576,820,93359,1118,90991
7,경기도,성남시분당구,404133,250683,104254,140622,1266,197,265,2677,249281,1402,153450
8,경기도,의정부시,400177,187916,92465,89731,1241,259,584,1742,186022,1894,212261
9,경기도,안양시만안구,208936,111050,54630,53271,712,157,199,904,109873,1177,97886
10,경기도,안양시동안구,265101,155858,77230,74669,1001,163,186,1432,154681,1177,109243


In [82]:
# 수치형 열만 합계 구하기
summary_row = gyeonggi_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경기도')

# summary_row를 맨 위에 붙이기
gyeonggi_8th_with_total = pd.concat([summary_row, gyeonggi_8th], ignore_index=True)

In [83]:
gyeonggi_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김동연,득표수_2_국민의힘_김은혜,득표수_3_정의당_황순식,득표수_4_기본소득당_서태성,득표수_5_진보당_송영주,득표수_6_무소속_강용석,득표수_계,무효투표수,기권수
0,경기도,합계,11497206,5820631,2827593,2818680,38525,9314,13939,54758,5762809,57822,5676575
1,경기도,수원시장안구,199204,106092,53333,49817,627,153,259,869,105058,1034,93112
2,경기도,수원시권선구,352407,173856,88562,80304,1090,312,406,1580,172254,1602,178551
3,경기도,수원시팔달구,164086,79283,37937,38931,488,169,176,724,78425,858,84803
4,경기도,수원시영통구,296856,160556,81109,75184,1142,212,223,1647,159517,1039,136300
5,경기도,성남시수정구,208907,106922,53684,49669,630,250,493,964,105690,1232,101985
6,경기도,성남시중원구,185468,94477,49268,41860,623,212,576,820,93359,1118,90991
7,경기도,성남시분당구,404133,250683,104254,140622,1266,197,265,2677,249281,1402,153450
8,경기도,의정부시,400177,187916,92465,89731,1241,259,584,1742,186022,1894,212261
9,경기도,안양시만안구,208936,111050,54630,53271,712,157,199,904,109873,1177,97886


In [84]:
gyeonggi_8th_with_total.to_csv("temp1_governor_gyeonggi_8.csv", index=False, encoding="utf-8-sig")


## Gangwon


In [85]:
gangwon_8th = dfs_8th['강원도']
gangwon_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,강원도,춘천시,NaN,NaN,NaN,NaN,더불어민주당\n이광재,국민의힘\n김진태,\n,\n,\n,\n,NaN,NaN,NaN
1,강원도,춘천시,합계,NaN,"244,406","137,765","63,714","72,126",0,0,0,0,"135,840","1,925","106,641"
2,강원도,원주시,합계,NaN,"304,060","157,538","77,841","77,298",0,0,0,0,"155,139","2,399","146,522"
3,강원도,강릉시,합계,NaN,"185,804","101,944","42,113","58,225",0,0,0,0,"100,338","1,606","83,860"
4,강원도,동해시,합계,NaN,"76,886","42,047","17,499","23,731",0,0,0,0,"41,230",817,"34,839"
5,강원도,삼척시,합계,NaN,"57,023","37,549","15,807","20,819",0,0,0,0,"36,626",923,"19,474"
6,강원도,태백시,합계,NaN,"35,236","22,538","10,288","11,722",0,0,0,0,"22,010",528,"12,698"
7,강원도,정선군,합계,NaN,"32,001","22,082","10,570","10,767",0,0,0,0,"21,337",745,"9,919"
8,강원도,속초시,합계,NaN,"71,621","37,094","16,266","20,124",0,0,0,0,"36,390",704,"34,527"
9,강원도,고성군,합계,NaN,"24,776","16,776","7,164","9,024",0,0,0,0,"16,188",588,"8,000"


In [86]:
gangwon_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [87]:
rename_gangwon = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_이광재',
    '후보2': '득표수_2_국민의힘_김진태',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [88]:
gangwon_8th = gangwon_8th.rename(columns=rename_gangwon).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
gangwon_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이광재,득표수_2_국민의힘_김진태,득표수_계,무효투표수,기권수
1,강원도,춘천시,"244,406","137,765","63,714","72,126","135,840","1,925","106,641"
2,강원도,원주시,"304,060","157,538","77,841","77,298","155,139","2,399","146,522"
3,강원도,강릉시,"185,804","101,944","42,113","58,225","100,338","1,606","83,860"
4,강원도,동해시,"76,886","42,047","17,499","23,731","41,230",817,"34,839"
5,강원도,삼척시,"57,023","37,549","15,807","20,819","36,626",923,"19,474"
6,강원도,태백시,"35,236","22,538","10,288","11,722","22,010",528,"12,698"
7,강원도,정선군,"32,001","22,082","10,570","10,767","21,337",745,"9,919"
8,강원도,속초시,"71,621","37,094","16,266","20,124","36,390",704,"34,527"
9,강원도,고성군,"24,776","16,776","7,164","9,024","16,188",588,"8,000"
10,강원도,양양군,"25,359","16,671","6,749","9,445","16,194",477,"8,688"


In [89]:
gangwon_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 1 to 18
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                18 non-null     object
 1   구시군               18 non-null     object
 2   선거인수              18 non-null     object
 3   투표수               18 non-null     object
 4   득표수_1_더불어민주당_이광재  18 non-null     object
 5   득표수_2_국민의힘_김진태    18 non-null     object
 6   득표수_계             18 non-null     object
 7   무효투표수             18 non-null     object
 8   기권수               18 non-null     object
dtypes: object(9)
memory usage: 1.4+ KB


In [90]:
gangwon_8th = gangwon_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gangwon_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이광재,득표수_2_국민의힘_김진태,득표수_계,무효투표수,기권수
1,강원도,춘천시,244406,137765,63714,72126,135840,1925,106641
2,강원도,원주시,304060,157538,77841,77298,155139,2399,146522
3,강원도,강릉시,185804,101944,42113,58225,100338,1606,83860
4,강원도,동해시,76886,42047,17499,23731,41230,817,34839
5,강원도,삼척시,57023,37549,15807,20819,36626,923,19474
6,강원도,태백시,35236,22538,10288,11722,22010,528,12698
7,강원도,정선군,32001,22082,10570,10767,21337,745,9919
8,강원도,속초시,71621,37094,16266,20124,36390,704,34527
9,강원도,고성군,24776,16776,7164,9024,16188,588,8000
10,강원도,양양군,25359,16671,6749,9445,16194,477,8688


In [91]:
# 수치형 열만 합계 구하기
summary_row = gangwon_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '강원도')

# summary_row를 맨 위에 붙이기
gangwon_8th_with_total = pd.concat([summary_row, gangwon_8th], ignore_index=True)

In [92]:
gangwon_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이광재,득표수_2_국민의힘_김진태,득표수_계,무효투표수,기권수
0,강원도,합계,1336080,772498,347766,409461,757227,15271,563582
1,강원도,춘천시,244406,137765,63714,72126,135840,1925,106641
2,강원도,원주시,304060,157538,77841,77298,155139,2399,146522
3,강원도,강릉시,185804,101944,42113,58225,100338,1606,83860
4,강원도,동해시,76886,42047,17499,23731,41230,817,34839
5,강원도,삼척시,57023,37549,15807,20819,36626,923,19474
6,강원도,태백시,35236,22538,10288,11722,22010,528,12698
7,강원도,정선군,32001,22082,10570,10767,21337,745,9919
8,강원도,속초시,71621,37094,16266,20124,36390,704,34527
9,강원도,고성군,24776,16776,7164,9024,16188,588,8000


In [93]:
gangwon_8th_with_total.to_csv("temp1_governor_gangwon_8.csv", index=False, encoding="utf-8-sig")


## Chungbuk


In [94]:
chungbuk_8th = dfs_8th['충청북도']
chungbuk_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,충청북도,청주시상당구,NaN,NaN,NaN,NaN,더불어민주당\n노영민,국민의힘\n김영환,\n,\n,\n,\n,NaN,NaN,NaN
1,충청북도,청주시상당구,합계,NaN,"163,058","79,612","33,296","45,312",0,0,0,0,"78,608","1,004","83,446"
2,충청북도,청주시서원구,합계,NaN,"165,453","80,317","33,934","45,411",0,0,0,0,"79,345",972,"85,136"
3,충청북도,청주시흥덕구,합계,NaN,"223,618","98,308","42,745","54,369",0,0,0,0,"97,114","1,194","125,310"
4,충청북도,청주시청원구,합계,NaN,"160,395","72,662","32,455","39,209",0,0,0,0,"71,664",998,"87,733"
5,충청북도,충주시,합계,NaN,"181,044","89,723","34,286","53,640",0,0,0,0,"87,926","1,797","91,321"
6,충청북도,제천시,합계,NaN,"115,563","62,806","23,887","37,865",0,0,0,0,"61,752","1,054","52,757"
7,충청북도,단양군,합계,NaN,"25,692","17,427","6,235","10,779",0,0,0,0,"17,014",413,"8,265"
8,충청북도,영동군,합계,NaN,"41,123","27,150","10,070","16,251",0,0,0,0,"26,321",829,"13,973"
9,충청북도,보은군,합계,NaN,"28,963","19,543","7,199","11,692",0,0,0,0,"18,891",652,"9,420"


In [95]:
chungbuk_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [96]:
rename_chungbuk = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_노영민',
    '후보2': '득표수_2_국민의힘_김영환',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [97]:
chungbuk_8th = chungbuk_8th.rename(columns=rename_chungbuk).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
chungbuk_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_노영민,득표수_2_국민의힘_김영환,득표수_계,무효투표수,기권수
1,충청북도,청주시상당구,"163,058","79,612","33,296","45,312","78,608","1,004","83,446"
2,충청북도,청주시서원구,"165,453","80,317","33,934","45,411","79,345",972,"85,136"
3,충청북도,청주시흥덕구,"223,618","98,308","42,745","54,369","97,114","1,194","125,310"
4,충청북도,청주시청원구,"160,395","72,662","32,455","39,209","71,664",998,"87,733"
5,충청북도,충주시,"181,044","89,723","34,286","53,640","87,926","1,797","91,321"
6,충청북도,제천시,"115,563","62,806","23,887","37,865","61,752","1,054","52,757"
7,충청북도,단양군,"25,692","17,427","6,235","10,779","17,014",413,"8,265"
8,충청북도,영동군,"41,123","27,150","10,070","16,251","26,321",829,"13,973"
9,충청북도,보은군,"28,963","19,543","7,199","11,692","18,891",652,"9,420"
10,충청북도,옥천군,"44,686","28,846","11,716","16,235","27,951",895,"15,840"


In [98]:
chungbuk_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 1 to 14
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                14 non-null     object
 1   구시군               14 non-null     object
 2   선거인수              14 non-null     object
 3   투표수               14 non-null     object
 4   득표수_1_더불어민주당_노영민  14 non-null     object
 5   득표수_2_국민의힘_김영환    14 non-null     object
 6   득표수_계             14 non-null     object
 7   무효투표수             14 non-null     object
 8   기권수               14 non-null     object
dtypes: object(9)
memory usage: 1.1+ KB


In [99]:
chungbuk_8th = chungbuk_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
chungbuk_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_노영민,득표수_2_국민의힘_김영환,득표수_계,무효투표수,기권수
1,충청북도,청주시상당구,163058,79612,33296,45312,78608,1004,83446
2,충청북도,청주시서원구,165453,80317,33934,45411,79345,972,85136
3,충청북도,청주시흥덕구,223618,98308,42745,54369,97114,1194,125310
4,충청북도,청주시청원구,160395,72662,32455,39209,71664,998,87733
5,충청북도,충주시,181044,89723,34286,53640,87926,1797,91321
6,충청북도,제천시,115563,62806,23887,37865,61752,1054,52757
7,충청북도,단양군,25692,17427,6235,10779,17014,413,8265
8,충청북도,영동군,41123,27150,10070,16251,26321,829,13973
9,충청북도,보은군,28963,19543,7199,11692,18891,652,9420
10,충청북도,옥천군,44686,28846,11716,16235,27951,895,15840


In [100]:
# 수치형 열만 합계 구하기
summary_row = chungbuk_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청북도')

# summary_row를 맨 위에 붙이기
chungbuk_8th_with_total = pd.concat([summary_row, chungbuk_8th], ignore_index=True)

In [101]:
chungbuk_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_노영민,득표수_2_국민의힘_김영환,득표수_계,무효투표수,기권수
0,충청북도,합계,1368779,692324,284166,395517,679683,12641,676455
1,충청북도,청주시상당구,163058,79612,33296,45312,78608,1004,83446
2,충청북도,청주시서원구,165453,80317,33934,45411,79345,972,85136
3,충청북도,청주시흥덕구,223618,98308,42745,54369,97114,1194,125310
4,충청북도,청주시청원구,160395,72662,32455,39209,71664,998,87733
5,충청북도,충주시,181044,89723,34286,53640,87926,1797,91321
6,충청북도,제천시,115563,62806,23887,37865,61752,1054,52757
7,충청북도,단양군,25692,17427,6235,10779,17014,413,8265
8,충청북도,영동군,41123,27150,10070,16251,26321,829,13973
9,충청북도,보은군,28963,19543,7199,11692,18891,652,9420


In [102]:
chungbuk_8th_with_total.to_csv("temp1_governor_chungbuk_8.csv", index=False, encoding="utf-8-sig")


## Chungnam


In [103]:
chungnam_8th = dfs_8th['충청남도']
chungnam_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,충청남도,천안시서북구,NaN,NaN,NaN,NaN,더불어민주당\n양승조,국민의힘\n김태흠,\n,\n,\n,\n,NaN,NaN,NaN
1,충청남도,천안시서북구,합계,NaN,"290,313","122,388","62,212","57,890",0,0,0,0,"120,102","2,286","167,925"
2,충청남도,천안시동남구,합계,NaN,"257,709","109,108","51,262","55,733",0,0,0,0,"106,995","2,113","148,601"
3,충청남도,공주시,합계,NaN,"91,847","51,461","21,615","28,250",0,0,0,0,"49,865","1,596","40,386"
4,충청남도,보령시,합계,NaN,"86,264","52,539","18,976","31,729",0,0,0,0,"50,705","1,834","33,725"
5,충청남도,아산시,합계,NaN,"268,765","119,352","57,114","59,468",0,0,0,0,"116,582","2,770","149,413"
6,충청남도,서산시,합계,NaN,"148,744","72,245","32,258","37,456",0,0,0,0,"69,714","2,531","76,499"
7,충청남도,태안군,합계,NaN,"55,435","34,715","14,522","18,185",0,0,0,0,"32,707","2,008","20,720"
8,충청남도,금산군,합계,NaN,"44,747","28,678","12,853","14,512",0,0,0,0,"27,365","1,313","16,069"
9,충청남도,논산시,합계,NaN,"99,942","53,814","24,238","27,506",0,0,0,0,"51,744","2,070","46,128"


In [104]:
chungnam_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [105]:
rename_chungnam = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_양승조',
    '후보2': '득표수_2_국민의힘_김태흠',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [106]:
chungnam_8th = chungnam_8th.rename(columns=rename_chungnam).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
chungnam_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_국민의힘_김태흠,득표수_계,무효투표수,기권수
1,충청남도,천안시서북구,"290,313","122,388","62,212","57,890","120,102","2,286","167,925"
2,충청남도,천안시동남구,"257,709","109,108","51,262","55,733","106,995","2,113","148,601"
3,충청남도,공주시,"91,847","51,461","21,615","28,250","49,865","1,596","40,386"
4,충청남도,보령시,"86,264","52,539","18,976","31,729","50,705","1,834","33,725"
5,충청남도,아산시,"268,765","119,352","57,114","59,468","116,582","2,770","149,413"
6,충청남도,서산시,"148,744","72,245","32,258","37,456","69,714","2,531","76,499"
7,충청남도,태안군,"55,435","34,715","14,522","18,185","32,707","2,008","20,720"
8,충청남도,금산군,"44,747","28,678","12,853","14,512","27,365","1,313","16,069"
9,충청남도,논산시,"99,942","53,814","24,238","27,506","51,744","2,070","46,128"
10,충청남도,계룡시,"34,875","19,307","8,666","10,218","18,884",423,"15,568"


In [107]:
chungnam_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 1 to 16
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                16 non-null     object
 1   구시군               16 non-null     object
 2   선거인수              16 non-null     object
 3   투표수               16 non-null     object
 4   득표수_1_더불어민주당_양승조  16 non-null     object
 5   득표수_2_국민의힘_김태흠    16 non-null     object
 6   득표수_계             16 non-null     object
 7   무효투표수             16 non-null     object
 8   기권수               16 non-null     object
dtypes: object(9)
memory usage: 1.3+ KB


In [108]:
chungnam_8th = chungnam_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
chungnam_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_국민의힘_김태흠,득표수_계,무효투표수,기권수
1,충청남도,천안시서북구,290313,122388,62212,57890,120102,2286,167925
2,충청남도,천안시동남구,257709,109108,51262,55733,106995,2113,148601
3,충청남도,공주시,91847,51461,21615,28250,49865,1596,40386
4,충청남도,보령시,86264,52539,18976,31729,50705,1834,33725
5,충청남도,아산시,268765,119352,57114,59468,116582,2770,149413
6,충청남도,서산시,148744,72245,32258,37456,69714,2531,76499
7,충청남도,태안군,55435,34715,14522,18185,32707,2008,20720
8,충청남도,금산군,44747,28678,12853,14512,27365,1313,16069
9,충청남도,논산시,99942,53814,24238,27506,51744,2070,46128
10,충청남도,계룡시,34875,19307,8666,10218,18884,423,15568


In [109]:
# 수치형 열만 합계 구하기
summary_row = chungnam_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청남도')

# summary_row를 맨 위에 붙이기
chungnam_8th_with_total = pd.concat([summary_row, chungnam_8th], ignore_index=True)

In [110]:
chungnam_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_국민의힘_김태흠,득표수_계,무효투표수,기권수
0,충청남도,합계,1803096,898369,401308,468658,869966,28403,904727
1,충청남도,천안시서북구,290313,122388,62212,57890,120102,2286,167925
2,충청남도,천안시동남구,257709,109108,51262,55733,106995,2113,148601
3,충청남도,공주시,91847,51461,21615,28250,49865,1596,40386
4,충청남도,보령시,86264,52539,18976,31729,50705,1834,33725
5,충청남도,아산시,268765,119352,57114,59468,116582,2770,149413
6,충청남도,서산시,148744,72245,32258,37456,69714,2531,76499
7,충청남도,태안군,55435,34715,14522,18185,32707,2008,20720
8,충청남도,금산군,44747,28678,12853,14512,27365,1313,16069
9,충청남도,논산시,99942,53814,24238,27506,51744,2070,46128


In [111]:
chungnam_8th_with_total.to_csv("temp1_governor_chungnam_8.csv", index=False, encoding="utf-8-sig")


## Jeonbuk


In [112]:
jeonbuk_8th = dfs_8th['전라북도']
jeonbuk_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,전라북도,전주시완산구,NaN,NaN,NaN,NaN,더불어민주당\n김관영,국민의힘\n조배숙,\n,\n,\n,\n,NaN,NaN,NaN
1,전라북도,전주시완산구,합계,NaN,"311,508","125,313","99,377","23,509",0,0,0,0,"122,886","2,427","186,195"
2,전라북도,전주시덕진구,합계,NaN,"238,934","96,938","75,741","19,377",0,0,0,0,"95,118","1,820","141,996"
3,전라북도,군산시,합계,NaN,"224,926","87,125","73,109","12,003",0,0,0,0,"85,112","2,013","137,801"
4,전라북도,익산시,합계,NaN,"239,077","107,245","83,134","20,682",0,0,0,0,"103,816","3,429","131,832"
5,전라북도,정읍시,합계,NaN,"93,307","54,283","44,026","7,960",0,0,0,0,"51,986","2,297","39,024"
6,전라북도,남원시,합계,NaN,"69,007","44,516","35,477","7,046",0,0,0,0,"42,523","1,993","24,491"
7,전라북도,김제시,합계,NaN,"72,358","42,668","33,765","7,114",0,0,0,0,"40,879","1,789","29,690"
8,전라북도,완주군,합계,NaN,"78,284","41,408","32,573","7,144",0,0,0,0,"39,717","1,691","36,876"
9,전라북도,진안군,합계,NaN,"22,634","16,616","13,062","2,796",0,0,0,0,"15,858",758,"6,018"


In [113]:
jeonbuk_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [114]:
rename_jeonbuk = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_김관영',
    '후보2': '득표수_2_국민의힘_조배숙',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [115]:
jeonbuk_8th = jeonbuk_8th.rename(columns=rename_jeonbuk).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
jeonbuk_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김관영,득표수_2_국민의힘_조배숙,득표수_계,무효투표수,기권수
1,전라북도,전주시완산구,"311,508","125,313","99,377","23,509","122,886","2,427","186,195"
2,전라북도,전주시덕진구,"238,934","96,938","75,741","19,377","95,118","1,820","141,996"
3,전라북도,군산시,"224,926","87,125","73,109","12,003","85,112","2,013","137,801"
4,전라북도,익산시,"239,077","107,245","83,134","20,682","103,816","3,429","131,832"
5,전라북도,정읍시,"93,307","54,283","44,026","7,960","51,986","2,297","39,024"
6,전라북도,남원시,"69,007","44,516","35,477","7,046","42,523","1,993","24,491"
7,전라북도,김제시,"72,358","42,668","33,765","7,114","40,879","1,789","29,690"
8,전라북도,완주군,"78,284","41,408","32,573","7,144","39,717","1,691","36,876"
9,전라북도,진안군,"22,634","16,616","13,062","2,796","15,858",758,"6,018"
10,전라북도,무주군,"21,279","16,268","11,971","3,381","15,352",916,"5,011"


In [116]:
jeonbuk_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 1 to 15
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                15 non-null     object
 1   구시군               15 non-null     object
 2   선거인수              15 non-null     object
 3   투표수               15 non-null     object
 4   득표수_1_더불어민주당_김관영  15 non-null     object
 5   득표수_2_국민의힘_조배숙    15 non-null     object
 6   득표수_계             15 non-null     object
 7   무효투표수             15 non-null     object
 8   기권수               15 non-null     object
dtypes: object(9)
memory usage: 1.2+ KB


In [117]:
jeonbuk_8th = jeonbuk_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
jeonbuk_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김관영,득표수_2_국민의힘_조배숙,득표수_계,무효투표수,기권수
1,전라북도,전주시완산구,311508,125313,99377,23509,122886,2427,186195
2,전라북도,전주시덕진구,238934,96938,75741,19377,95118,1820,141996
3,전라북도,군산시,224926,87125,73109,12003,85112,2013,137801
4,전라북도,익산시,239077,107245,83134,20682,103816,3429,131832
5,전라북도,정읍시,93307,54283,44026,7960,51986,2297,39024
6,전라북도,남원시,69007,44516,35477,7046,42523,1993,24491
7,전라북도,김제시,72358,42668,33765,7114,40879,1789,29690
8,전라북도,완주군,78284,41408,32573,7144,39717,1691,36876
9,전라북도,진안군,22634,16616,13062,2796,15858,758,6018
10,전라북도,무주군,21279,16268,11971,3381,15352,916,5011


In [118]:
# 수치형 열만 합계 구하기
summary_row = jeonbuk_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '전라북도')

# summary_row를 맨 위에 붙이기
jeonbuk_8th_with_total = pd.concat([summary_row, jeonbuk_8th], ignore_index=True)

In [119]:
jeonbuk_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김관영,득표수_2_국민의힘_조배숙,득표수_계,무효투표수,기권수
0,전라북도,합계,1532133,745354,591510,128828,720338,25016,786779
1,전라북도,전주시완산구,311508,125313,99377,23509,122886,2427,186195
2,전라북도,전주시덕진구,238934,96938,75741,19377,95118,1820,141996
3,전라북도,군산시,224926,87125,73109,12003,85112,2013,137801
4,전라북도,익산시,239077,107245,83134,20682,103816,3429,131832
5,전라북도,정읍시,93307,54283,44026,7960,51986,2297,39024
6,전라북도,남원시,69007,44516,35477,7046,42523,1993,24491
7,전라북도,김제시,72358,42668,33765,7114,40879,1789,29690
8,전라북도,완주군,78284,41408,32573,7144,39717,1691,36876
9,전라북도,진안군,22634,16616,13062,2796,15858,758,6018


In [120]:
jeonbuk_8th_with_total.to_csv("temp1_governor_jeonbuk_8.csv", index=False, encoding="utf-8-sig")


## Jeonnam


In [121]:
jeonnam_8th = dfs_8th['전라남도']
jeonnam_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,전라남도,목포시,NaN,NaN,NaN,NaN,더불어민주당\n김영록,국민의힘\n이정현,진보당\n민점기,\n,\n,\n,NaN,NaN,NaN
1,전라남도,목포시,합계,NaN,"183,412","96,968","76,408","13,045","4,489",0,0,0,"93,942","3,026","86,444"
2,전라남도,여수시,합계,NaN,"236,881","109,135","85,684","16,912","4,180",0,0,0,"106,776","2,359","127,746"
3,전라남도,순천시,합계,NaN,"235,432","128,163","79,776","40,082","5,472",0,0,0,"125,330","2,833","107,269"
4,전라남도,나주시,합계,NaN,"98,951","53,036","39,317","8,638","2,984",0,0,0,"50,939","2,097","45,915"
5,전라남도,광양시,합계,NaN,"126,604","69,108","47,884","15,322","4,079",0,0,0,"67,285","1,823","57,496"
6,전라남도,담양군,합계,NaN,"41,720","26,611","19,641","4,358","1,615",0,0,0,"25,614",997,"15,109"
7,전라남도,장성군,합계,NaN,"38,470","26,499","19,721","3,946","1,429",0,0,0,"25,096","1,403","11,971"
8,전라남도,곡성군,합계,NaN,"25,196","18,643","9,549","7,225",859,0,0,0,"17,633","1,010","6,553"
9,전라남도,구례군,합계,NaN,"22,848","17,417","11,783","3,662","1,095",0,0,0,"16,540",877,"5,431"


In [122]:
jeonnam_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [123]:
rename_jeonnam = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_김영록',
    '후보2': '득표수_2_국민의힘_이정현',
    '후보3': '득표수_4_진보당_민점기',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [124]:
jeonnam_8th = jeonnam_8th.rename(columns=rename_jeonnam).drop(columns=['읍면동명', '구분', '후보4', '후보5', '후보6']).drop(index=0)
jeonnam_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_2_국민의힘_이정현,득표수_4_진보당_민점기,득표수_계,무효투표수,기권수
1,전라남도,목포시,"183,412","96,968","76,408","13,045","4,489","93,942","3,026","86,444"
2,전라남도,여수시,"236,881","109,135","85,684","16,912","4,180","106,776","2,359","127,746"
3,전라남도,순천시,"235,432","128,163","79,776","40,082","5,472","125,330","2,833","107,269"
4,전라남도,나주시,"98,951","53,036","39,317","8,638","2,984","50,939","2,097","45,915"
5,전라남도,광양시,"126,604","69,108","47,884","15,322","4,079","67,285","1,823","57,496"
6,전라남도,담양군,"41,720","26,611","19,641","4,358","1,615","25,614",997,"15,109"
7,전라남도,장성군,"38,470","26,499","19,721","3,946","1,429","25,096","1,403","11,971"
8,전라남도,곡성군,"25,196","18,643","9,549","7,225",859,"17,633","1,010","6,553"
9,전라남도,구례군,"22,848","17,417","11,783","3,662","1,095","16,540",877,"5,431"
10,전라남도,고흥군,"57,371","44,658","32,273","6,793","2,424","41,490","3,168","12,713"


In [125]:
jeonnam_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 1 to 22
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                22 non-null     object
 1   구시군               22 non-null     object
 2   선거인수              22 non-null     object
 3   투표수               22 non-null     object
 4   득표수_1_더불어민주당_김영록  22 non-null     object
 5   득표수_2_국민의힘_이정현    22 non-null     object
 6   득표수_4_진보당_민점기     22 non-null     object
 7   득표수_계             22 non-null     object
 8   무효투표수             22 non-null     object
 9   기권수               22 non-null     object
dtypes: object(10)
memory usage: 1.8+ KB


In [126]:
jeonnam_8th = jeonnam_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
jeonnam_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_2_국민의힘_이정현,득표수_4_진보당_민점기,득표수_계,무효투표수,기권수
1,전라남도,목포시,183412,96968,76408,13045,4489,93942,3026,86444
2,전라남도,여수시,236881,109135,85684,16912,4180,106776,2359,127746
3,전라남도,순천시,235432,128163,79776,40082,5472,125330,2833,107269
4,전라남도,나주시,98951,53036,39317,8638,2984,50939,2097,45915
5,전라남도,광양시,126604,69108,47884,15322,4079,67285,1823,57496
6,전라남도,담양군,41720,26611,19641,4358,1615,25614,997,15109
7,전라남도,장성군,38470,26499,19721,3946,1429,25096,1403,11971
8,전라남도,곡성군,25196,18643,9549,7225,859,17633,1010,6553
9,전라남도,구례군,22848,17417,11783,3662,1095,16540,877,5431
10,전라남도,고흥군,57371,44658,32273,6793,2424,41490,3168,12713


In [127]:
# 수치형 열만 합계 구하기
summary_row = jeonnam_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '전라남도')

# summary_row를 맨 위에 붙이기
jeonnam_8th_with_total = pd.concat([summary_row, jeonnam_8th], ignore_index=True)

In [128]:
jeonnam_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_2_국민의힘_이정현,득표수_4_진보당_민점기,득표수_계,무효투표수,기권수
0,전라남도,합계,1580098,923347,672433,167020,48336,887789,35558,656751
1,전라남도,목포시,183412,96968,76408,13045,4489,93942,3026,86444
2,전라남도,여수시,236881,109135,85684,16912,4180,106776,2359,127746
3,전라남도,순천시,235432,128163,79776,40082,5472,125330,2833,107269
4,전라남도,나주시,98951,53036,39317,8638,2984,50939,2097,45915
5,전라남도,광양시,126604,69108,47884,15322,4079,67285,1823,57496
6,전라남도,담양군,41720,26611,19641,4358,1615,25614,997,15109
7,전라남도,장성군,38470,26499,19721,3946,1429,25096,1403,11971
8,전라남도,곡성군,25196,18643,9549,7225,859,17633,1010,6553
9,전라남도,구례군,22848,17417,11783,3662,1095,16540,877,5431


In [129]:
jeonnam_8th_with_total.to_csv("temp1_governor_jeonnam_8.csv", index=False, encoding="utf-8-sig")


## Gyeongbuk


In [130]:
gyeongbuk_8th = dfs_8th['경상북도']
gyeongbuk_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,경상북도,포항시북구,NaN,NaN,NaN,NaN,더불어민주당\n임미애,국민의힘\n이철우,\n,\n,\n,\n,NaN,NaN,NaN
1,경상북도,포항시북구,합계,NaN,"230,960","105,786","23,150","80,784",0,0,0,0,"103,934","1,852","125,174"
2,경상북도,포항시남구,합계,NaN,"196,727","90,434","20,195","68,137",0,0,0,0,"88,332","2,102","106,293"
3,경상북도,울릉군,합계,NaN,"8,339","6,795","1,217","5,270",0,0,0,0,"6,487",308,"1,544"
4,경상북도,경주시,합계,NaN,"220,490","109,647","22,875","83,997",0,0,0,0,"106,872","2,775","110,843"
5,경상북도,김천시,합계,NaN,"120,471","67,277","13,915","51,776",0,0,0,0,"65,691","1,586","53,194"
6,경상북도,안동시,합계,NaN,"135,862","74,540","20,435","51,965",0,0,0,0,"72,400","2,140","61,322"
7,경상북도,구미시,합계,NaN,"337,510","144,547","34,597","108,033",0,0,0,0,"142,630","1,917","192,963"
8,경상북도,영주시,합계,NaN,"89,061","56,297","12,783","41,642",0,0,0,0,"54,425","1,872","32,764"
9,경상북도,영천시,합계,NaN,"90,932","51,707","9,328","40,517",0,0,0,0,"49,845","1,862","39,225"


In [131]:
gyeongbuk_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [132]:
rename_gyeongbuk = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_임미애',
    '후보2': '득표수_2_국민의힘_이철우',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [133]:
gyeongbuk_8th = gyeongbuk_8th.rename(columns=rename_gyeongbuk).drop(columns=['읍면동명', '구분', '후보3', '후보4', '후보5', '후보6']).drop(index=0)
gyeongbuk_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임미애,득표수_2_국민의힘_이철우,득표수_계,무효투표수,기권수
1,경상북도,포항시북구,"230,960","105,786","23,150","80,784","103,934","1,852","125,174"
2,경상북도,포항시남구,"196,727","90,434","20,195","68,137","88,332","2,102","106,293"
3,경상북도,울릉군,"8,339","6,795","1,217","5,270","6,487",308,"1,544"
4,경상북도,경주시,"220,490","109,647","22,875","83,997","106,872","2,775","110,843"
5,경상북도,김천시,"120,471","67,277","13,915","51,776","65,691","1,586","53,194"
6,경상북도,안동시,"135,862","74,540","20,435","51,965","72,400","2,140","61,322"
7,경상북도,구미시,"337,510","144,547","34,597","108,033","142,630","1,917","192,963"
8,경상북도,영주시,"89,061","56,297","12,783","41,642","54,425","1,872","32,764"
9,경상북도,영천시,"90,932","51,707","9,328","40,517","49,845","1,862","39,225"
10,경상북도,상주시,"84,980","54,367","9,992","42,260","52,252","2,115","30,613"


In [134]:
gyeongbuk_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 1 to 24
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                24 non-null     object
 1   구시군               24 non-null     object
 2   선거인수              24 non-null     object
 3   투표수               24 non-null     object
 4   득표수_1_더불어민주당_임미애  24 non-null     object
 5   득표수_2_국민의힘_이철우    24 non-null     object
 6   득표수_계             24 non-null     object
 7   무효투표수             24 non-null     object
 8   기권수               24 non-null     object
dtypes: object(9)
memory usage: 1.8+ KB


In [135]:
gyeongbuk_8th = gyeongbuk_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gyeongbuk_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임미애,득표수_2_국민의힘_이철우,득표수_계,무효투표수,기권수
1,경상북도,포항시북구,230960,105786,23150,80784,103934,1852,125174
2,경상북도,포항시남구,196727,90434,20195,68137,88332,2102,106293
3,경상북도,울릉군,8339,6795,1217,5270,6487,308,1544
4,경상북도,경주시,220490,109647,22875,83997,106872,2775,110843
5,경상북도,김천시,120471,67277,13915,51776,65691,1586,53194
6,경상북도,안동시,135862,74540,20435,51965,72400,2140,61322
7,경상북도,구미시,337510,144547,34597,108033,142630,1917,192963
8,경상북도,영주시,89061,56297,12783,41642,54425,1872,32764
9,경상북도,영천시,90932,51707,9328,40517,49845,1862,39225
10,경상북도,상주시,84980,54367,9992,42260,52252,2115,30613


In [136]:
# 수치형 열만 합계 구하기
summary_row = gyeongbuk_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경상북도')

# summary_row를 맨 위에 붙이기
gyeongbuk_8th_with_total = pd.concat([summary_row, gyeongbuk_8th], ignore_index=True)

In [137]:
gyeongbuk_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임미애,득표수_2_국민의힘_이철우,득표수_계,무효투표수,기권수
0,경상북도,합계,2268707,1194595,255775,904675,1160450,34145,1074112
1,경상북도,포항시북구,230960,105786,23150,80784,103934,1852,125174
2,경상북도,포항시남구,196727,90434,20195,68137,88332,2102,106293
3,경상북도,울릉군,8339,6795,1217,5270,6487,308,1544
4,경상북도,경주시,220490,109647,22875,83997,106872,2775,110843
5,경상북도,김천시,120471,67277,13915,51776,65691,1586,53194
6,경상북도,안동시,135862,74540,20435,51965,72400,2140,61322
7,경상북도,구미시,337510,144547,34597,108033,142630,1917,192963
8,경상북도,영주시,89061,56297,12783,41642,54425,1872,32764
9,경상북도,영천시,90932,51707,9328,40517,49845,1862,39225


In [138]:
gyeongbuk_8th_with_total.to_csv("temp1_governor_gyeongbuk_8.csv", index=False, encoding="utf-8-sig")


## Gyeongnam


In [139]:
gyeongnam_8th = dfs_8th['경상남도']
gyeongnam_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,경상남도,창원시의창구,NaN,NaN,NaN,NaN,더불어민주당\n양문석,국민의힘\n박완수,정의당\n여영국,통일한국당\n최진석,\n,\n,NaN,NaN,NaN
1,경상남도,창원시의창구,합계,NaN,"185,184","94,824","23,944","64,989","4,366",370,0,0,"93,669","1,155","90,360"
2,경상남도,창원시성산구,합계,NaN,"212,811","112,825","29,902","67,977","13,523",412,0,0,"111,814","1,011","99,986"
3,경상남도,창원시마산합포구,합계,NaN,"156,850","83,924","19,970","59,943","2,417",330,0,0,"82,660","1,264","72,926"
4,경상남도,창원시마산회원구,합계,NaN,"159,983","85,597","22,063","59,376","2,601",324,0,0,"84,364","1,233","74,386"
5,경상남도,창원시진해구,합계,NaN,"159,730","75,210","22,186","48,889","2,687",298,0,0,"74,060","1,150","84,520"
6,경상남도,진주시,합계,NaN,"292,168","155,185","42,182","104,561","4,812",999,0,0,"152,554","2,631","136,983"
7,경상남도,통영시,합계,NaN,"106,064","61,244","18,160","40,307","1,081",473,0,0,"60,021","1,223","44,820"
8,경상남도,고성군,합계,NaN,"44,745","30,306","7,918","20,130",889,381,0,0,"29,318",988,"14,439"
9,경상남도,사천시,합계,NaN,"93,946","55,534","12,887","38,414","1,917",803,0,0,"54,021","1,513","38,412"


In [140]:
gyeongnam_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [141]:
rename_gyeongnam = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_양문석',
    '후보2': '득표수_2_국민의힘_박완수',
    '후보3': '득표수_3_정의당_여영국',
    '후보4': '득표수_4_통일한국당_최진석',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [142]:
gyeongnam_8th = gyeongnam_8th.rename(columns=rename_gyeongnam).drop(columns=['읍면동명', '구분', '후보5', '후보6']).drop(index=0)
gyeongnam_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양문석,득표수_2_국민의힘_박완수,득표수_3_정의당_여영국,득표수_4_통일한국당_최진석,득표수_계,무효투표수,기권수
1,경상남도,창원시의창구,"185,184","94,824","23,944","64,989","4,366",370,"93,669","1,155","90,360"
2,경상남도,창원시성산구,"212,811","112,825","29,902","67,977","13,523",412,"111,814","1,011","99,986"
3,경상남도,창원시마산합포구,"156,850","83,924","19,970","59,943","2,417",330,"82,660","1,264","72,926"
4,경상남도,창원시마산회원구,"159,983","85,597","22,063","59,376","2,601",324,"84,364","1,233","74,386"
5,경상남도,창원시진해구,"159,730","75,210","22,186","48,889","2,687",298,"74,060","1,150","84,520"
6,경상남도,진주시,"292,168","155,185","42,182","104,561","4,812",999,"152,554","2,631","136,983"
7,경상남도,통영시,"106,064","61,244","18,160","40,307","1,081",473,"60,021","1,223","44,820"
8,경상남도,고성군,"44,745","30,306","7,918","20,130",889,381,"29,318",988,"14,439"
9,경상남도,사천시,"93,946","55,534","12,887","38,414","1,917",803,"54,021","1,513","38,412"
10,경상남도,김해시,"444,484","203,399","78,054","115,747","6,166",824,"200,791","2,608","241,085"


In [143]:
gyeongnam_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 1 to 22
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                22 non-null     object
 1   구시군               22 non-null     object
 2   선거인수              22 non-null     object
 3   투표수               22 non-null     object
 4   득표수_1_더불어민주당_양문석  22 non-null     object
 5   득표수_2_국민의힘_박완수    22 non-null     object
 6   득표수_3_정의당_여영국     22 non-null     object
 7   득표수_4_통일한국당_최진석   22 non-null     object
 8   득표수_계             22 non-null     object
 9   무효투표수             22 non-null     object
 10  기권수               22 non-null     object
dtypes: object(11)
memory usage: 2.0+ KB


In [144]:
gyeongnam_8th = gyeongnam_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
gyeongnam_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양문석,득표수_2_국민의힘_박완수,득표수_3_정의당_여영국,득표수_4_통일한국당_최진석,득표수_계,무효투표수,기권수
1,경상남도,창원시의창구,185184,94824,23944,64989,4366,370,93669,1155,90360
2,경상남도,창원시성산구,212811,112825,29902,67977,13523,412,111814,1011,99986
3,경상남도,창원시마산합포구,156850,83924,19970,59943,2417,330,82660,1264,72926
4,경상남도,창원시마산회원구,159983,85597,22063,59376,2601,324,84364,1233,74386
5,경상남도,창원시진해구,159730,75210,22186,48889,2687,298,74060,1150,84520
6,경상남도,진주시,292168,155185,42182,104561,4812,999,152554,2631,136983
7,경상남도,통영시,106064,61244,18160,40307,1081,473,60021,1223,44820
8,경상남도,고성군,44745,30306,7918,20130,889,381,29318,988,14439
9,경상남도,사천시,93946,55534,12887,38414,1917,803,54021,1513,38412
10,경상남도,김해시,444484,203399,78054,115747,6166,824,200791,2608,241085


In [145]:
# 수치형 열만 합계 구하기
summary_row = gyeongnam_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경상남도')

# summary_row를 맨 위에 붙이기
gyeongnam_8th_with_total = pd.concat([summary_row, gyeongnam_8th], ignore_index=True)

In [146]:
gyeongnam_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양문석,득표수_2_국민의힘_박완수,득표수_3_정의당_여영국,득표수_4_통일한국당_최진석,득표수_계,무효투표수,기권수
0,경상남도,합계,2804287,1497400,431569,963473,58933,12353,1466328,31072,1306887
1,경상남도,창원시의창구,185184,94824,23944,64989,4366,370,93669,1155,90360
2,경상남도,창원시성산구,212811,112825,29902,67977,13523,412,111814,1011,99986
3,경상남도,창원시마산합포구,156850,83924,19970,59943,2417,330,82660,1264,72926
4,경상남도,창원시마산회원구,159983,85597,22063,59376,2601,324,84364,1233,74386
5,경상남도,창원시진해구,159730,75210,22186,48889,2687,298,74060,1150,84520
6,경상남도,진주시,292168,155185,42182,104561,4812,999,152554,2631,136983
7,경상남도,통영시,106064,61244,18160,40307,1081,473,60021,1223,44820
8,경상남도,고성군,44745,30306,7918,20130,889,381,29318,988,14439
9,경상남도,사천시,93946,55534,12887,38414,1917,803,54021,1513,38412


In [147]:
gyeongnam_8th_with_total.to_csv("temp1_governor_gyeongnam_8.csv", index=False, encoding="utf-8-sig")


## Jeju

In [148]:
jeju_8th = dfs_8th['제주특별자치도']
jeju_8th

,선거구명,구시군명,읍면동명,구분,선거인수_선거인수,투표수_선거인수,후보자별 득표수_후보1,후보2,후보3,후보4,후보5,후보6,계_선거인수,무효투표수_선거인수,기권수_선거인수
0,제주특별자치도,제주시,NaN,NaN,NaN,NaN,더불어민주당\n오영훈,국민의힘\n허향진,녹색당\n부순정,무소속\n박찬식,\n,\n,NaN,NaN,NaN
1,제주특별자치도,제주시,합계,NaN,"409,110","216,419","118,762","83,539","4,389","6,878",0,0,"213,568","2,851","192,691"
2,제주특별자치도,서귀포시,합계,NaN,"155,974","83,720","44,354","33,247","1,361","3,260",0,0,"82,222","1,498","72,254"


In [149]:
jeju_8th.columns.tolist()

['선거구명',
 '구시군명',
 '읍면동명',
 '구분',
 '선거인수_선거인수',
 '투표수_선거인수',
 '후보자별 득표수_후보1',
 '후보2',
 '후보3',
 '후보4',
 '후보5',
 '후보6',
 '계_선거인수',
 '무효투표수_선거인수',
 '기권수_선거인수']

In [150]:
rename_jeju = {
    '선거구명': '시도',
    '구시군명': '구시군',
    '선거인수_선거인수': '선거인수',
    '투표수_선거인수': '투표수',
    '후보자별 득표수_후보1': '득표수_1_더불어민주당_오영훈',
    '후보2': '득표수_2_국민의힘_허향진',
    '후보3': '득표수_4_녹색당_부순정',
    '후보4': '득표수_5_무소속_박찬식',
    '계_선거인수': '득표수_계',
    '무효투표수_선거인수': '무효투표수',
    '기권수_선거인수': '기권수'
}

In [151]:
jeju_8th = jeju_8th.rename(columns=rename_jeju).drop(columns=['읍면동명', '구분', '후보5', '후보6']).drop(index=0)
jeju_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오영훈,득표수_2_국민의힘_허향진,득표수_4_녹색당_부순정,득표수_5_무소속_박찬식,득표수_계,무효투표수,기권수
1,제주특별자치도,제주시,"409,110","216,419","118,762","83,539","4,389","6,878","213,568","2,851","192,691"
2,제주특별자치도,서귀포시,"155,974","83,720","44,354","33,247","1,361","3,260","82,222","1,498","72,254"


In [152]:
jeju_8th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 1 to 2
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                2 non-null      object
 1   구시군               2 non-null      object
 2   선거인수              2 non-null      object
 3   투표수               2 non-null      object
 4   득표수_1_더불어민주당_오영훈  2 non-null      object
 5   득표수_2_국민의힘_허향진    2 non-null      object
 6   득표수_4_녹색당_부순정     2 non-null      object
 7   득표수_5_무소속_박찬식     2 non-null      object
 8   득표수_계             2 non-null      object
 9   무효투표수             2 non-null      object
 10  기권수               2 non-null      object
dtypes: object(11)
memory usage: 308.0+ bytes


In [153]:
jeju_8th = jeju_8th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)
jeju_8th

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오영훈,득표수_2_국민의힘_허향진,득표수_4_녹색당_부순정,득표수_5_무소속_박찬식,득표수_계,무효투표수,기권수
1,제주특별자치도,제주시,409110,216419,118762,83539,4389,6878,213568,2851,192691
2,제주특별자치도,서귀포시,155974,83720,44354,33247,1361,3260,82222,1498,72254


In [154]:
# 수치형 열만 합계 구하기
summary_row = jeju_8th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '제주특별자치도')

# summary_row를 맨 위에 붙이기
jeju_8th_with_total = pd.concat([summary_row, jeju_8th], ignore_index=True)

In [155]:
jeju_8th_with_total

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오영훈,득표수_2_국민의힘_허향진,득표수_4_녹색당_부순정,득표수_5_무소속_박찬식,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,565084,300139,163116,116786,5750,10138,295790,4349,264945
1,제주특별자치도,제주시,409110,216419,118762,83539,4389,6878,213568,2851,192691
2,제주특별자치도,서귀포시,155974,83720,44354,33247,1361,3260,82222,1498,72254


In [156]:
jeju_8th_with_total.to_csv("temp1_governor_jeju_8.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [157]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp1_governor_jeonbuk_8.csv
Added: temp1_governor_sejong_8.csv
Added: temp1_governor_gyeonggi_8.csv
Added: temp1_governor_incheon_8.csv
Added: temp1_governor_ulsan_8.csv
Added: temp1_governor_gyeongnam_8.csv
Added: temp1_governor_chungnam_8.csv
Added: temp1_governor_seoul_8.csv
Added: temp1_governor_daegu_8.csv
Added: temp1_governor_gwangju_8.csv
Added: temp1_governor_jeonnam_8.csv
Added: temp1_governor_gangwon_8.csv
Added: temp1_governor_daejeon_8.csv
Added: temp1_governor_busan_8.csv
Added: temp1_governor_gyeongbuk_8.csv
Added: temp1_governor_jeju_8.csv
Added: temp1_governor_chungbuk_8.csv
Total 17 files compressed.
